# 04 Extracción de datos en la web

## Caso: Datos del Clima

El objetivo de esta actividad es abordar un caso en el que se requiere extraer información que se encuentra en la web, en particular para este ejercicio nos enfocaremos en extraer datos sobre el clima desde una página web para almacenarlos en un archivo CSV. De esta forma podríamos generar un histórico del clima para regiones y espacios temporales de interés para aplicaciones como predicción de tráfico vial, etc.

Al concluir nuestra actividad se espera tener un archivo llamado `clima_7_dias_Boulder.csv` que contenga los siguientes datos para cada día:

* period
* short_desc
* temp
* desc

Visualmente, esperamos llegar a construir la siguiente tabla de datos:
<img src="images\\target.PNG">



## Descripción de la actividad

***Descargar datos metereológicos***

Para iniciar con el desarrollo de la actividad se proponen los siguientes pasos para descargar la página y comenzar el análisis:

1. Instalar o verificar que se tengan instalas las librerías: requests y bs4
2. Cargar a memoria la página web: https://forecast.weather.gov/MapClick.php?lat=40.0466&lon=-105.2523# .YwpRBy2B1f0
3. Crear una instancia de la clase `BeatifulSoup` para analizar la página
4. Buscar el `div` que tenga un `id` = `seven-day-forecast`
5. Dentro del `div`encontremos cada elemento de pronósticos individual
6. Para verificar que vamos bien, imprimamos el primer elemento del pronóstivo

In [1]:
#pip install requests
#pip install bs4

import requests
from bs4 import BeautifulSoup

url = "https://forecast.weather.gov/MapClick.php?lat=40.0466&lon=-105.2523#.YwpRBy2B1f0"

In [2]:
page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

seven_day = soup.find(id='seven-day-forecast')

forecast_items = seven_day.find_all(class_ = 'tombstone-container')

In [3]:
forecast_items

[<div class="tombstone-container"><p class="period-name">Today</p><p><img alt="Today: Sunny, with a high near 52. Calm wind becoming northeast 5 to 8 mph in the afternoon. " class="forecast-icon" src="newimages/medium/few.png" title="Today: Sunny, with a high near 52. Calm wind becoming northeast 5 to 8 mph in the afternoon. "/></p><p class="temp temp-high">High: 52 °F</p><p class="short-desc">Sunny</p></div>,
 <div class="tombstone-container"><p class="period-name">Tonight</p><p><img alt="Tonight: Mostly clear, with a low around 30. North wind around 5 mph becoming calm. " class="forecast-icon" src="newimages/medium/nfew.png" title="Tonight: Mostly clear, with a low around 30. North wind around 5 mph becoming calm. "/></p><p class="temp temp-low">Low: 30 °F</p><p class="short-desc">Mostly Clear</p></div>,
 <div class="tombstone-container"><p class="period-name">Friday</p><p><img alt="Friday: Sunny, with a high near 59. West southwest wind 3 to 8 mph. " class="forecast-icon" src="newim

In [4]:
tonight = forecast_items[1]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
 </p>
 <p>
  <img alt="Tonight: Mostly clear, with a low around 30. North wind around 5 mph becoming calm. " class="forecast-icon" src="newimages/medium/nfew.png" title="Tonight: Mostly clear, with a low around 30. North wind around 5 mph becoming calm. "/>
 </p>
 <p class="temp temp-low">
  Low: 30 °F
 </p>
 <p class="short-desc">
  Mostly Clear
 </p>
</div>



***Extracción de la información de esta noche***

¿Cómo podemos ver dentro del elemento de pronóstico de esta noche toda la información que queremos? Hay cuatro elementos de información que podemos extraer:

* El nombre del elemento del pronóstico.
* La descripción de las condiciones.
* Una breve descripción de las condiciones.
* La temperatura máxima.

Primero extraeremos el nombre del elemento de pronóstico, la breve descripción y la temperatura, ya que todos son similares:

In [7]:
#code
period = tonight.find(class_="period-name").get_text()
print(period)

short_desc = tonight.find(class_="short-desc").get_text()
print(short_desc)

temp = tonight.find(class_="temp").get_text()
print(temp)

img = tonight.find("img")
desc = img['title']
print(desc)

Tonight
Mostly Clear
Low: 30 °F
Tonight: Mostly clear, with a low around 30. North wind around 5 mph becoming calm. 


Ahora, procedamos a extraer la descripción de las condiciones.

#code

***Extracción de toda la información***


Ahora que sabemos cómo extraer cada dato de forma individual, procedamos a extraer toda la información de una vez.

In [13]:
#periods
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
#print(periods)

#short_descs
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
#print(short_descs) 

#temps
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
#print(temps) 

#descs
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
#print(descs) 


['Today: Sunny, with a high near 52. Calm wind becoming northeast 5 to 8 mph in the afternoon. ', 'Tonight: Mostly clear, with a low around 30. North wind around 5 mph becoming calm. ', 'Friday: Sunny, with a high near 59. West southwest wind 3 to 8 mph. ', 'Friday Night: A chance of rain before 11pm, then a chance of rain and snow.  Mostly cloudy, with a low around 31. West southwest wind 6 to 8 mph.  Chance of precipitation is 50%. Little or no snow accumulation expected. ', 'Saturday: A 20 percent chance of snow before 11am.  Mostly sunny, with a high near 47. West northwest wind 3 to 8 mph. Winds could gust as high as 20 mph. ', 'Saturday Night: A 30 percent chance of snow after 11pm.  Mostly cloudy, with a low around 18.', 'Sunday: A 20 percent chance of snow.  Mostly sunny, with a high near 38.', 'Sunday Night: Partly cloudy, with a low around 16.', 'Monday: A slight chance of snow.  Mostly sunny, with a high near 36.']


***Construcción del DataFrame***

Ahora podemos combinar los datos en un `DataFrame`  de Pandas para posteriormente analizarlos. Recordemos que un `DataFrame` es un objeto que puede almacenar datos tabulares, lo que facilita el análisis de datos.